In [1]:
from flask import request, Response
from flask import Flask
import pprint
from config import netbox_api
from types import SimpleNamespace

In [2]:
from nornir import InitNornir
from credentials import(netbox_url,
                        netbox_token,
                        device_username,
                        device_password)
from nornir_napalm.plugins.tasks import napalm_get
def create_nornir_session():
    """ 
    Инициализируем nornir, но для "hosts" используем данные из netbox
    :return: nr_session
    """
    nr_session = InitNornir(
        inventory={
            "plugin": "NetBoxInventory2",
            "options": {
                "nb_url": netbox_url,
                "nb_token": netbox_token,
                "group_file": "./inventory/groups.yml",
                "defaults_file": "./inventory/defaults.yml",
            },
        },
    )
    return nr_session

In [ ]:
def test():
    # Получаем данные через flask от webhook netbox:
    mng_int_id = request.json['data']['id'] # type: ignore
    get_device_interface = netbox_api.dcim.interfaces.get(mng_int_id)
    device_id = get_device_interface.device.id # type: ignore
    d = dict(netbox_api.dcim.devices.get(device_id).config_context) # type: ignore
    pprint.pprint(d)
    config_context = SimpleNamespace(**d)
    #print(key for key, value in config_context.items())
    #print(config_context.device_username)

    nr = create_nornir_session()

    sw = nr.filter(hostname = '10.30.1.105') # производим отбор по конкретному хосту
    sw.inventory.hosts['SW-PLGN-1'].username = config_context.device_username
    sw.inventory.hosts['SW-PLGN-1'].password = config_context.device_password
    get_int = sw.run(task=napalm_get, getters=['get_interfaces'])
    print(get_int)
    
    return Response(status=204)

In [44]:
# api/ipam/prefixes/39/available-ips/
get_device_interface = netbox_api.dcim.interfaces.get(764)
print(get_device_interface)
intf = get_device_interface
if intf.untagged_vlan and intf.mac_address:
    vlan_id = intf.untagged_vlan.vid
    
    device = netbox_api.dcim.devices.get(intf.device.id)
   
    if device.primary_ip == False:
        
        get_available_ips = netbox_api.ipam.prefixes.get(vlan_vid = vlan_id).available_ips.list()
        
        pprint.pprint(get_available_ips[0])

    
changes = dict()
#changes = dict.fromkeys(config_context.interface, None)
#changes['id'] = 117
changes['address'] = str(get_available_ips[0])
changes['status'] = 'dhcp'
changes['assigned_object_type'] = 'dcim.interface'
changes['assigned_object_id'] = intf.id
changes['dns_name'] = str(device.name)
changes['description'] = str(device.name)
#netbox_api.dcim.interfaces.update([changes])
#print('Clear {} netbox interface config'.format(device_intf))
netbox_api.ipam.ip_addresses.create([changes])
device.update([{"primary_ip":str(get_available_ips[0])}])
print(changes)

Ethernet 0


AttributeError: 'list' object has no attribute 'items'

In [ ]:
# Create a Flask instance
app = Flask(__name__)
"""
Webhook POST
    Name:'Fixed IP into DHCPd'
    Content types: 'IPAM > IP Address'
"""
app.add_url_rule("/api/fixed_ip",
                 methods=["POST"],
                 view_func=test)
"""
Webhook POST
    Name:'Change the cable'
    Content types: 'DCIM > Cable'
"""
app.add_url_rule("/api/cable_change",
                methods=['POST'],
                view_func=test)
"""
Webhook POST
    Name:'Update the interface'
    Content types: 'DCIM > Interfaces'
"""
app.add_url_rule("/api/int_update",
                 methods=['POST'],
                 view_func=test)
    
if __name__ == "__main__": 
    app.run(host='0.0.0.0', port=8080)